<a href="https://colab.research.google.com/github/KangJ0n0/Datmin/blob/main/Scraping%20%2B%20clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq google-play-scraper

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from google_play_scraper import Sort, reviews, app
import nltk
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
path = "gdrive/MyDrive/"

Mounted at /content/gdrive


In [ ]:
app_packages = [
    'blibli.mobile.commerce',
]

In [ ]:
app_infos = []

for application in tqdm (app_packages):
  info =  app (application, lang = 'id', country = 'id')
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  5.41it/s]


In [ ]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list (range(1,6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews (
          ap,
          lang = 'id',
          country = 'id',
          sort=sort_order,
          count=200 if score == 3 else 100,
          filter_score_with = score
      )
      for r in rvs :
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:03<00:00,  3.99s/it]


In [ ]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.encode('ascii', 'ignore').decode('ascii')  # Remove emojis and non-ASCII characters
    return text

In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)

In [ ]:
app_reviews_df = app_reviews_df.sort_values(["appId","at"])

In [ ]:
app_reviews_df.to_csv (path+'hasilreviewstest.csv', index = None, header=True)